In [1]:
import numpy as np
import cv2 as cv
import trimesh
import matplotlib.pyplot as plt


from visualisation import visualize_results, visualize_results_mesh, visualize_results_colored
from check_visibility import get_visible_vertices, load_image_info, prepare_camera_and_pose_data, load_and_prepare_mesh
from check_visibility import IMG_FOLDER_PATH, IMG_FORMAT, POSES_FOLDER, MESH_PATH
import cv2 as cv

ImportError: cannot import name 'visualize_results_colored' from 'visualisation' (/mnt/942A3A4F2A3A2F1C/Work/Bridges_Project/Bridge-Selection/visualisation.py)

In [2]:
image_name = 'DJI_20240417190632_0129_Z'
pose_info, camera_info = load_image_info(image_name)
camera_matrix, distortion_coefficients, rotation, center, img_height, img_width = prepare_camera_and_pose_data(camera_info, pose_info)
pier, pier_cutted_ = load_and_prepare_mesh(MESH_PATH, camera_matrix, rotation, center, img_height, img_width)

visible_indices_orig, visible_indices = get_visible_vertices(image_name)


Total vertices 398
Projected vertices 282
Ray hits 227
Visible 88
Total vertices original 14172


In [3]:
visualize_results_mesh(pier_cutted_, visible_indices, rotation, center)

# Use 10 images

In [4]:
images_list =   ['DJI_20240418185755_0185_Z',
                'DJI_20240418185757_0186_Z',
                'DJI_20240418185759_0187_Z',
                'DJI_20240418185801_0188_Z',
                'DJI_20240418185803_0189_Z',
                'DJI_20240418185805_0190_Z',
                'DJI_20240418185807_0191_Z',
                'DJI_20240418185809_0192_Z',
                'DJI_20240418185811_0193_Z',
                'DJI_20240418185813_0194_Z',
                'DJI_20240418185815_0195_Z']


visibility_dict = {n: get_visible_vertices(n, verbose=False)[0] for n in images_list}
visibility_list = list(visibility_dict.values())

vertex_count = np.zeros(len(pier.vertices), dtype=int)
for vis in visibility_list:
    vertex_count[vis] += 1

In [32]:
import plotly.express as px
def visualize_results_colored(mesh, visible_count, camera_position,):
   
    if len(camera_position.shape)>1:
        camera_position = camera_position.squeeze()
    
    fig = px.scatter_3d(
        x=mesh.vertices[:, 0], y=mesh.vertices[:, 1], z=mesh.vertices[:, 2],
        color=visible_count, size=visible_count+1
    )
 
    fig.show()

In [33]:
visualize_results_colored(pier, vertex_count, center)

In [39]:
p = sum(vertex_count>0)/len(vertex_count)*100
print(f'Percentage of visible vertices {p:.2f}%')

Percentage of visible vertices 1.70%


# Use all poses

In [48]:
import os

# file_format = IMG_FORMAT
# folder = IMG_FOLDER_PATH
file_format = '.json'
folder = 'poses'

def get_names_without_extension(folder_path):
    files = os.listdir(folder_path)
    image_names = [os.path.splitext(f)[0] for f in files if f.lower().endswith(file_format.lower())]
    return image_names

image_names = get_names_without_extension(folder)
print(len(image_names))

5689


In [51]:
visibility_dict = {n: get_visible_vertices(n, verbose=False)[0] for n in image_names}
visibility_list = list(visibility_dict.values())

vertex_count = np.zeros(len(pier.vertices), dtype=int)
for vis in visibility_list:
    vertex_count[vis] += 1

AssertionError: Image not found

In [50]:
visualize_results_colored(pier, vertex_count, center)